In [1]:
import warnings
import re
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import pandas as pd
df = pd.read_csv('C:\\Ashish\\Project\\dataset\\trainset1.csv')#,nrows = 1500)
df.text[:5]

0    #weather Another #fakenews video @CNN Anderson...
1    Staring Down Hurricane Florence via NASA https...
2    #hurricane #florence http:// tra.one/tcphurFlo...
3    Hurricane Florence is still moving through the...
4    Houston is collecting donations for Hurricane ...
Name: text, dtype: object

In [3]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))   

# remove twitter handles (@user)
df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")
    
for i in range(len(df)):
    tweet = df.iloc[i,df.columns.get_loc('text')]
    
    tweet = tweet.lower()
    
    
    tweet = re.sub(r"\S+\/.\S+", "", tweet
                   
    
    tweet = re.sub(r'[^\w\s]','', tweet)
    
    word_tokens = word_tokenize(tweet) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    tweet = ' '.join(filtered_sentence)
    
    tweet = tweet.strip()
    df.iloc[i,df.columns.get_loc('text')] = tweet
df.text[0:5]

SyntaxError: invalid syntax (<ipython-input-4-e176b189e7d3>, line 17)

In [ ]:
#Common Word Removal
# freq = pd.Series(' '.join(df['text']).split()).value_counts()[:10]
# freq

# freq = list(freq.index)
# df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# df['text'].head()

In [ ]:
# Rare Word Removal
# freq = pd.Series(' '.join(df['text']).split()).value_counts()[-10:]
# freq

# freq = list(freq.index)
# df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# df['text'].head()

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df['text'].head()

In [ ]:
from io import StringIO

col = ['category', 'text']
df = df[col]
df = df[pd.notnull(df['text'])]

df.columns = ['category', 'text']

df['c_id'] = df['category'].factorize()[0]
category_id_df = df[['category', 'c_id']].drop_duplicates().sort_values('c_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['c_id', 'category']].values)
df.head()

In [ ]:
for string in df.text:
    print(string)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('category').text.count().plot.bar(ylim=0)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df.text).toarray()
labels = df.c_id
features.shape

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['category'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
print(clf.predict(count_vect.transform(["Florence save animals"])))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

import seaborn as sns

sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
model = LinearSVC()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.category.values, yticklabels=category_id_df.category.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
model = MultinomialNB()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.category.values, yticklabels=category_id_df.category.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from IPython.display import display

for predicted in category_id_df.c_id:
  for actual in category_id_df.c_id:
    if predicted != actual and conf_mat[actual, predicted] >= 10:
      print("'{}' predicted as '{}' : {} examples.".format(id_to_category[actual], id_to_category[predicted], conf_mat[actual, predicted]))
      display(df.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['category', 'text']])
      print('')

In [ ]:
df.loc[175].text

In [ ]:
model.fit(features, labels)

N = 2
for Product, category_id in sorted(category_to_id.items()):
  indices = np.argsort(model.coef_[category_id])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
  bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
  print("# '{}':".format(Product))
  print("  . Top unigrams:\n       . {}".format('\n       . '.join(unigrams)))
  print("  . Top bigrams:\n       . {}".format('\n       . '.join(bigrams)))

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, target_names = df['category'].unique()))